# classifier.deploySQL

In [ ]:
classifier.deploySQL(predict: bool = False)

Returns the SQL code needed to deploy the model. 

### Parameters

<table id="parameters">
    <tr> <th>Name</th> <th>Type</th> <th>Optional</th> <th>Description</th> </tr>
    <tr> <td><div class="param_name">predict</div></td> <td><div class="type">bool</div></td> <td><div class = "yes">&#10003;</div></td> <td>If set to True, returns the prediction instead of the probability.</td> </tr>
</table>

### Returns

<b>str / list</b> : the SQL code needed to deploy the model.

### Example

In [51]:
from vertica_ml_python.learn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
model.fit("public.iris", 
          ["PetalLengthCm", "PetalWidthCm", "SepalWidthCm"], 
          "Species")
print(model.deploySQL())

(SELECT row_id, "PetalLengthCm", "PetalWidthCm", "SepalWidthCm", "Species", predict_knc, COUNT(*) / 5 AS proba_predict FROM (SELECT x."PetalLengthCm", x."PetalWidthCm", x."SepalWidthCm", ROW_NUMBER() OVER(PARTITION BY x."PetalLengthCm", x."PetalWidthCm", x."SepalWidthCm", row_id ORDER BY POWER(POWER(ABS(x."PetalLengthCm" - y."PetalLengthCm"), 2) + POWER(ABS(x."PetalWidthCm" - y."PetalWidthCm"), 2) + POWER(ABS(x."SepalWidthCm" - y."SepalWidthCm"), 2), 1 / 2)) AS ordered_distance, x."Species", y."Species" AS predict_knc, row_id FROM (SELECT *, ROW_NUMBER() OVER() AS row_id FROM public.iris WHERE "PetalLengthCm" IS NOT NULL AND "PetalWidthCm" IS NOT NULL AND "SepalWidthCm" IS NOT NULL) x CROSS JOIN (SELECT * FROM public.iris WHERE "PetalLengthCm" IS NOT NULL AND "PetalWidthCm" IS NOT NULL AND "SepalWidthCm" IS NOT NULL) y) z WHERE ordered_distance <= 5 GROUP BY "PetalLengthCm", "PetalWidthCm", "SepalWidthCm", "Species", row_id, predict_knc) knc_table


In [52]:
print(model.deploySQL(predict = True))

(SELECT "PetalLengthCm", "PetalWidthCm", "SepalWidthCm", "Species", predict_knc FROM (SELECT "PetalLengthCm", "PetalWidthCm", "SepalWidthCm", "Species", predict_knc, ROW_NUMBER() OVER (PARTITION BY "PetalLengthCm", "PetalWidthCm", "SepalWidthCm" ORDER BY proba_predict DESC) AS order_prediction FROM (SELECT row_id, "PetalLengthCm", "PetalWidthCm", "SepalWidthCm", "Species", predict_knc, COUNT(*) / 5 AS proba_predict FROM (SELECT x."PetalLengthCm", x."PetalWidthCm", x."SepalWidthCm", ROW_NUMBER() OVER(PARTITION BY x."PetalLengthCm", x."PetalWidthCm", x."SepalWidthCm", row_id ORDER BY POWER(POWER(ABS(x."PetalLengthCm" - y."PetalLengthCm"), 2) + POWER(ABS(x."PetalWidthCm" - y."PetalWidthCm"), 2) + POWER(ABS(x."SepalWidthCm" - y."SepalWidthCm"), 2), 1 / 2)) AS ordered_distance, x."Species", y."Species" AS predict_knc, row_id FROM (SELECT *, ROW_NUMBER() OVER() AS row_id FROM public.iris WHERE "PetalLengthCm" IS NOT NULL AND "PetalWidthCm" IS NOT NULL AND "SepalWidthCm" IS NOT NULL) x CROSS 